In [ ]:
import  sys
import  h5py
import  numpy              as np
import  pandas             as pd
import  tensorflow         as tf
import  gudhi              as gd
import  matplotlib.pyplot  as plt
%matplotlib notebook

from    features                         import * 
from    hyperopt                         import *
from    hpsklearn                        import *
from    sklearn.preprocessing            import *
from    sklearn.svm                      import *
from    sklearn.ensemble                 import *
from    sklearn.decomposition            import *
from    sklearn.linear_model             import *
from    sklearn.model_selection          import *
from    sklearn.pipeline                 import *
from    sklearn.manifold                 import *
from    sklearn.neighbors                import *
from    sklearn.metrics                  import *
from    sklearn.metrics.pairwise         import *
from    sklearn.feature_selection        import *
from    MKLpy.algorithms                 import *
from    mpl_toolkits.mplot3d             import Axes3D

sys.path.append("./layers/rips")
libPDR = tf.load_op_library("persistence_diagram_rips.so")
import _persistence_diagram_rips_grad

sys.path.append("./layers/simplicial")
libPDS = tf.load_op_library("persistence_diagram_simplicial_complex.so")
import _persistence_diagram_simplicial_complex_grad

Utils

In [ ]:
def pc_to_array(data):

    num_pc = len(data.keys())

    max_pts = 0
    for idx in data.keys():
        max_pts = max(max_pts, data[idx].shape[0])
    euclidean_dimension = data["0"].shape[1]

    dataset = np.full((num_pc, euclidean_dimension * max_pts), np.nan)

    for idx in range(num_pc):
        point_cloud = np.reshape(np.array(data[str(idx)]), [-1])
        dataset[idx, 0:len(point_cloud)] = point_cloud

    return dataset

# 0. Read data

Provide names of different tasks (as in the corresponding .csv file).

In [ ]:
tasks = [("material","classification"), ("sor","regression")]
for i in range(1,13):
    tasks.append(("bin "+str(i),"regression"))

Read train data.

In [ ]:
path_to_train_feat = "../../../../Documents/datasets/bridge/7/train.csv"
train_feat = pd.read_csv(path_to_train_feat)

In [ ]:
path_to_train_clouds = "../../../../Documents/datasets/bridge/7/train_pc.hdf5"
train_pc = pc_to_list(h5py.File(path_to_train_clouds, "r"))

Specify if test set is 1. a fraction of train set (False) / 2. a separate set with possibly missing labels (True).

In [ ]:
use_fraction_of_train_for_test = True

If test set is fraction of training set, specify ratio.

In [ ]:
test_size = 0.2

if use_fraction_of_train_for_test == True:
    train_num_pts = train_feat.shape[0]    
    perm = np.random.permutation(train_num_pts)
    limit = np.int(test_size * train_num_pts)
    test_sub, train_sub = np.sort(perm[:limit]), np.sort(perm[limit:]) #perm[:limit], perm[limit:]
    train_num_pts, test_num_pts = len(train_sub), len(test_sub)

Else, read test set.

In [ ]:
if use_fraction_of_train_for_test == False:
    path_to_test_feat = "../../../../Documents/datasets/bridge/7/test.csv"
    test_feat = pd.read_csv(path_to_test_feat)
    train_num_pts, test_num_pts = train_feat.shape[0], test_feat.shape[0]

In [ ]:
if use_fraction_of_train_for_test == False:
    path_to_test_clouds = "../../../../Documents/datasets/bridge/7/test_pc.hdf5"
    test_pc = pc_to_list(h5py.File(path_to_test_clouds, "r"))
    train_num_pts, test_num_pts = len(train_pc), len(test_pc)

Convert data frame into numpy array.

In [ ]:
train_F = np.array(train_feat)[:,1+len(tasks):]
num_features = train_F.shape[1]

if use_fraction_of_train_for_test == False:
    test_F = np.array(test_feat)[:,1+len(tasks):]

In [ ]:
list_train_pred, list_test_pred = [], []

Visualization.

In [ ]:
index = 1500

cloud = train_pc[index]
kde = KernelDensity(kernel="gaussian", bandwidth=0.7)
kde.fit(cloud)
vals = kde.score_samples(cloud)
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(cloud[:,0], cloud[:,1], cloud[:,2], c = vals, cmap ='seismic')
plt.show()

In [ ]:
train_feat.describe()

# 1. Deep Sets

Reset network.

In [ ]:
tf.reset_default_graph()
feed_train, feed_test, feed_epoch = dict(), dict(), dict()

Add labels.

In [ ]:
task_name, task_type = tasks[1]

ohe, le = OneHotEncoder(sparse = False), LabelEncoder()

train_full_labels = train_feat[task_name]
if use_fraction_of_train_for_test == True:
    train_labels = np.reshape(train_full_labels[train_sub], [-1,1])
    test_labels =  np.reshape(train_full_labels[test_sub], [-1,1])
else:
    train_labels = np.reshape(train_full_labels, [-1,1])
    
if task_type == "classification" or task_type == "metric":
    train_labels = ohe.fit_transform(np.reshape(le.fit_transform(train_labels), [-1,1]))
    if use_fraction_of_train_for_test == True:            
        test_labels = ohe.transform(np.reshape(le.transform(test_labels), [-1,1]))

num_labels = train_labels.shape[1]
label = tf.placeholder(tf.float32, shape = [None, num_labels], name = "labels")
feed_train[label] = train_labels

if use_fraction_of_train_for_test == True:
    feed_test[label] = test_labels
    
print("Task: " + task_name + " " + task_type + ", " + str(num_labels) + " label(s)\n")

### 1.1 Preprocessing

In [ ]:
N, input_dim = 1000, 3

preprocess = Pipeline([("Preprocessor",  DiagramPreprocessor(MinMaxScaler()))])

train_full_PC = preprocess.fit_transform(train_pc)
max_pts, num_pc = 0, len(train_full_PC)
for i in range(num_pc):
    max_pts = max(max_pts, train_full_PC[i].shape[0])
    
train_full_pad_PC, train_full_mask = np.zeros([num_pc, max_pts, input_dim]), np.zeros([num_pc, max_pts])
for i in range(num_pc):
    pc = train_full_PC[i]
    train_full_pad_PC[i,:pc.shape[0],:] = pc
    train_full_mask[i,:pc.shape[0]] = np.ones([pc.shape[0]])
    
if use_fraction_of_train_for_test == True:
    train_pad_PC, train_mask = train_full_pad_PC[train_sub,:], train_full_mask[train_sub,:]
    test_pad_PC, test_mask = train_full_pad_PC[test_sub,:], train_full_mask[test_sub,:]
else:
    train_pad_PC, train_mask = train_full_pad_PC, train_full_mask
    test_full_PC = preprocess.fit_transform(test_pc)
    num_pc_test = len(test_full_PC)
    test_pad_PC, test_mask = np.zeros([num_pc_test, max_pts, input_dim]), np.zeros([num_pc_test, max_pts])
    for i in range(num_pc_test):
        pc = test_full_PC[i]
        test_pad_PC[i,:min(max_pts,pc.shape[0]),:] = pc
        test_mask[i,:pc.shape[0]] = np.ones([pc.shape[0]])

### 1.2 Network

In [ ]:
cloud = tf.placeholder(tf.float32, shape = [None, N, 3], name = "point clouds")
mask = tf.placeholder(tf.float32, shape = [None, N], name = "masks")  
feed_test[cloud], feed_train[cloud], feed_test[mask], feed_train[mask] = test_pad_PC, train_pad_PC, test_m, train_m
    
# Phi network
with tf.variable_scope("phi"+str(i), reuse=tf.AUTO_REUSE):

    weight1 = tf.get_variable("w1", shape=[3,50], initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.1))
    biases1 = tf.get_variable("b1", shape=[1,50], initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.1))
    phi = tf.reshape(tf.tensordot(cloud, weight1, 1) + biases1, [-1,N,dim1])
    final_dim = 50
     
# Weight and mask
tiled_mask = tf.tile(tf.reshape(mask, [-1,N,1]), [1,1,final_dim])
masked_phi = tf.multiply(phi, tiled_mask)

# Permutation invariant op
vector = tf.reduce_sum(masked_phi, 1)

# Rho network
fn = tf.layers.dense(vector, num_labels)

### 1.3 Losses and optimization.

In [ ]:
if task_type == "classification":
    loss     = tf.reduce_mean(  tf.nn.softmax_cross_entropy_with_logits_v2(labels = label, logits = fn)  )
    accuracy = tf.reduce_mean(  tf.cast(tf.equal(tf.argmax(fn,1), tf.argmax(label,1)), dtype = tf.float32)  )
    tf.summary.scalar("accuracy", accuracy)
    
if task_type == "regression":
    loss = tf.losses.mean_squared_error(fn, label)
    
if task_type == "metric":
    fn_a, fn_b = tf.expand_dims(fn,0), tf.expand_dims(fn,1)
    l2diff = tf.reshape(tf.reduce_sum(tf.square(tf.add(fn_a,-fn_b)),2),[-1,1])
    lab = tf.argmax(label,1)
    lab_a, lab_b = tf.reshape(lab,[-1,1]), tf.reshape(lab,[1,-1])
    neg_pairs = tf.reshape(1.0-1.0*tf.cast(tf.equal(lab_a,lab_b), dtype=tf.float32), [-1,1])
    pos_pairs = 1.0-1.0*neg_pairs
    loss = tf.reduce_sum(tf.multiply(neg_pairs,tf.nn.relu(1.0-l2diff))) + tf.reduce_sum(tf.multiply(pos_pairs,l2diff))

tf.summary.scalar("loss", loss)

In [ ]:
opt = tf.train.RMSPropOptimizer(learning_rate=0.1)
train_step = opt.minimize(loss)

In [ ]:
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('./train', sess.graph)
test_writer = tf.summary.FileWriter('./test')

print("Variables = " + str(tf.trainable_variables()))
print("\nNumber of variables = " + str(np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()])))

### 1.4 Learning best predictor and evaluation on train and test sets.

In [ ]:
nb_epoch    = 100
print_every = 10
batch_size  = 320

with tf.Session() as sess:
    
    # Initialize parameters
    sess.run(tf.global_variables_initializer())
            
    # Score to print
    if task_type == "classification":
        score = accuracy
    if task_type == "regression" or task_type == "metric":
        score = loss
    
    # Learning best predictor
    for ep in range(nb_epoch):
            
        b = np.random.choice(train_num_pts, batch_size, replace = False)
        for k in feed_train.keys():
            feed_epoch[k] = feed_train[k][b]

        train_step.run(feed_dict = feed_epoch)
          
        if(ep % print_every == 0):
            train_summary, train_score = sess.run([merged, score], feed_dict = feed_train)
            train_writer.add_summary(train_summary, ep)
            test_summary, test_score = sess.run([merged, score], feed_dict = feed_test)
            test_writer.add_summary(test_summary, ep)
            print("\nTrain score on epoch " + str(ep) + " = " + str(train_score))
            if use_fraction_of_train_for_test == True:
                print("Test score on epoch " + str(ep) + "  = " + str(test_score))
            
    # Prediction
    train_pred, test_pred = sess.run(fn, feed_dict = feed_train), sess.run(fn, feed_dict = feed_test)
    if use_fraction_of_train_for_test == False:
        np.savetxt(task, test_pred)
          
    # Evaluation on train set
    if use_fraction_of_train_for_test == False:
        print("\nTrain score = " + str(sess.run(score, feed_dict = feed_train)))
        if task_type == "regression":
            plot_regression_result(train_labels, train_pred)
        if task_type == "classification":
            plot_confusion_matrix(confusion_matrix(np.argmax(train_labels,1), np.argmax(train_pred,1)))
        if task_type == "metric":
            dim_red = PCA(n_components = 3)
            red = dim_red.fit_transform(train_pred)
            trace = go.Scatter3d(x=red[:,0], y=red[:,1], z=red[:,2], mode="markers", 
                                 marker=dict(size=3, color=np.argmax(train_labels,1), colorscale="Viridis", opacity=0.8))
            py.offline.iplot({"data": [trace]})
        
    # Evaluation on test set
    else:
        print("Test score  = " + str(sess.run(score, feed_dict = feed_test)))
        if task_type == "regression":
            plot_regression_result(test_labels, test_pred)
        if task_type == "classification":
            plot_confusion_matrix(confusion_matrix(np.argmax(test_labels,1), np.argmax(test_pred,1)))
        if task_type == "metric":
            dim_red = PCA(n_components = 3)
            red = dim_red.fit_transform(test_pred)
            trace = go.Scatter3d(x=red[:,0], y=red[:,1], z=red[:,2], mode="markers", 
                                 marker=dict(size=3, color=np.argmax(test_labels,1), colorscale="Viridis", opacity=0.8))
            py.offline.iplot({"data": [trace]})